# 이진 판단: 천체의 펄서 여부 판정 신경망

In [3]:
# dataset : www.kaggle.com/pavanraj159/predicting-a-pulsar-star
# explanation : https://towering-payment-a36.notion.site/Chapter2-8d957bb3746d4bdd873eeca35f718303

In [17]:
import numpy as np
import csv

In [18]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

In [19]:
print(sigmoid(1.0))
print(sigmoid(-1000.0))

0.7310585786300049
0.0


<ipython-input-18-58db7051c8a7>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


## 구현 순서
- 메인 함수 정의
- 데이터 적재 함수 정의
- 이진 판단 후처리를 위한 순전파 및 역전파 함수 재정의
- 정확도 계산함수 재정의
- 시그모이드 관련 함수 정의

In [20]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001


def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD,[input_cnt, output_cnt])
    bias = np.zeros([output_cnt])


def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))
    
def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]

def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy

def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= LEARNING_RATE * G_w
    bias -= LEARNING_RATE * G_b

In [33]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

    
def load_pulsar_dataset():
    with open('../jch/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')
       
        
def forward_postproc(output, y):
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    loss = np.mean(entropy)
    return loss, [y, output, entropy]


def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)
    
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output


def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))


def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)


def eval_accuracy(output, y):
    estimate = np.greater(output, 0)
    answer = np.greater(y, 0.5)
    correct = np.equal(estimate, answer)
    
    return np.mean(correct)


def relu(x):
    return np.maximum(x, 0)


def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
    
def sigmoid_derv(x, y):
    return y * (1 - y)


def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))


def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [34]:
pulsar_exec()

Epoch 1: loss=0.134, accuracy=0.962/0.963
Epoch 2: loss=0.129, accuracy=0.967/0.972
Epoch 3: loss=0.124, accuracy=0.967/0.971
Epoch 4: loss=0.127, accuracy=0.968/0.968
Epoch 5: loss=0.134, accuracy=0.967/0.975
Epoch 6: loss=0.130, accuracy=0.967/0.974
Epoch 7: loss=0.129, accuracy=0.969/0.972
Epoch 8: loss=0.120, accuracy=0.968/0.975
Epoch 9: loss=0.126, accuracy=0.969/0.975
Epoch 10: loss=0.130, accuracy=0.969/0.957

Final Test: final accuracy = 0.957


In [35]:
data

array([[140.5625    ,  55.68378   ,  -0.23457141, ...,   7.9755316 ,
         74.242226  ,   0.        ],
       [102.50781   ,  58.88243   ,   0.46531814, ...,  10.576487  ,
        127.39358   ,   0.        ],
       [103.015625  ,  39.34165   ,   0.32332838, ...,   7.735822  ,
         63.17191   ,   0.        ],
       ...,
       [119.33594   ,  59.93594   ,   0.1593631 , ...,   2.4995172 ,
          4.595173  ,   0.        ],
       [114.50781   ,  53.9024    ,   0.20116138, ...,  10.007967  ,
        134.2389    ,   0.        ],
       [ 57.0625    ,  85.79734   ,   1.406391  , ...,  -1.5975266 ,
          1.4294753 ,   0.        ]], dtype=float32)

In [36]:
data.shape

(17898, 9)

In [43]:
def pulsar_exec_ext(epoch_count=10, mb_size=10, report=1, adjust_ratio=False):
    load_pulsar_dataset_ext(adjust_ratio)
    init_model()
    train_and_test_ext(epoch_count, mb_size, report)
    
    
def load_pulsar_dataset_ext(adjust_ratio):
    pulsars, stars = [], []
    with open('../jch/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')
        
        
def eval_accuracy(output, y):
    est_yes = np.greater(output, 0)
    ans_yes = np.greater(y, 0.5)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+fn)
    f1 = 2 * safe_div(recall*precision, recall+precision)
    
    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q


def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses = []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, _ = run_train(train_x, train_y)
            losses.append(loss)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            acc_str = ','.join(['%5.3f']*4) % tuple(acc)
            print('Epoch {}: loss={:5.3f}, result={}'. \
                  format(epoch+1, np.mean(losses), acc_str))
            
    acc = run_test(test_x, test_y)
    acc_str = ','.join(['%5.3f']*4) % tuple(acc)
    print('\nFinal Test: final result = {}'.format(acc_str))

In [44]:
pulsar_exec_ext()

Epoch 1: loss=0.132, result=0.972,0.938,0.746,0.831
Epoch 2: loss=0.141, result=0.956,0.967,0.535,0.689
Epoch 3: loss=0.145, result=0.972,0.948,0.731,0.826
Epoch 4: loss=0.123, result=0.975,0.905,0.813,0.857
Epoch 5: loss=0.132, result=0.972,0.945,0.737,0.828
Epoch 6: loss=0.131, result=0.975,0.925,0.792,0.853
Epoch 7: loss=0.132, result=0.971,0.963,0.713,0.819
Epoch 8: loss=0.132, result=0.976,0.923,0.807,0.861
Epoch 9: loss=0.137, result=0.967,0.930,0.688,0.791
Epoch 10: loss=0.127, result=0.976,0.914,0.810,0.859

Final Test: final result = 0.976,0.914,0.810,0.859


In [47]:
data

array([[140.5625    ,  55.68378067,  -0.23457141, ...,   7.97553158,
         74.24222565,   0.        ],
       [102.5078125 ,  58.88243103,   0.46531814, ...,  10.57648659,
        127.39357758,   0.        ],
       [103.015625  ,  39.3416481 ,   0.32332838, ...,   7.7358222 ,
         63.17190933,   0.        ],
       ...,
       [ 48.3671875 ,  30.057724  ,   4.50091219, ...,   0.740556  ,
         -0.60604012,   1.        ],
       [ 65.0703125 ,  34.73317719,   2.26607513, ...,   3.77811432,
         14.68210793,   1.        ],
       [ 84.0078125 ,  34.25239563,   1.36355782, ...,   4.08432388,
         15.84995556,   1.        ]])

In [48]:
data.shape

(32518, 9)

In [49]:
pulsar_exec_ext(adjust_ratio=True)

Epoch 1: loss=0.408, result=0.916,0.972,0.859,0.912
Epoch 2: loss=0.380, result=0.911,0.911,0.914,0.913
Epoch 3: loss=0.368, result=0.722,0.649,0.980,0.781
Epoch 4: loss=0.373, result=0.909,0.983,0.834,0.902
Epoch 5: loss=0.363, result=0.915,0.921,0.910,0.915
Epoch 6: loss=0.366, result=0.904,0.893,0.921,0.907
Epoch 7: loss=0.380, result=0.914,0.970,0.856,0.910
Epoch 8: loss=0.384, result=0.911,0.908,0.917,0.913
Epoch 9: loss=0.357, result=0.910,0.986,0.833,0.903
Epoch 10: loss=0.344, result=0.925,0.958,0.892,0.924

Final Test: final result = 0.925,0.958,0.892,0.924


In [50]:
data

array([[140.5625    ,  55.68378067,  -0.23457141, ...,   7.97553158,
         74.24222565,   0.        ],
       [102.5078125 ,  58.88243103,   0.46531814, ...,  10.57648659,
        127.39357758,   0.        ],
       [103.015625  ,  39.3416481 ,   0.32332838, ...,   7.7358222 ,
         63.17190933,   0.        ],
       ...,
       [ 48.3671875 ,  30.057724  ,   4.50091219, ...,   0.740556  ,
         -0.60604012,   1.        ],
       [ 65.0703125 ,  34.73317719,   2.26607513, ...,   3.77811432,
         14.68210793,   1.        ],
       [ 84.0078125 ,  34.25239563,   1.36355782, ...,   4.08432388,
         15.84995556,   1.        ]])

In [51]:
data.shape

(32518, 9)